In [4]:
import json
from openai import OpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored
from dotenv import load_dotenv
import os

API_KEY = os.getenv("API_KEY")
os.environ["OPENAI_API_KEY"] = API_KEY


GPT_MODEL = "gpt-4"
client = OpenAI()

In [5]:
student_1_description = "David Nguyen is a sophomore majoring in computer science at Stanford University. He is Asian American and has a 3.8 GPA. David is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after graduating."

# A simple prompt to extract information from "student_description" in a JSON format.
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''

In [6]:
prompt1

"\nPlease extract the following information from the given text and return it as a JSON object:\n\nname\nmajor\nschool\ngrades\nclub\n\nThis is the body of text to extract the information from:\nDavid Nguyen is a sophomore majoring in computer science at Stanford University. He is Asian American and has a 3.8 GPA. David is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after graduating.\n"

In [7]:
# Generating response back from gpt-3.5-turbo
openai_response = client.chat.completions.create(
    model = 'gpt-3.5-turbo',
    messages = [{'role': 'user', 'content': prompt1}]
)

openai_response.choices[0].message.content

'{\n  "name": "David Nguyen",\n  "major": "computer science",\n  "school": "Stanford University",\n  "grades": 3.8,\n  "club": "Robotics Club"\n}'

In [8]:
import json

json_response = json.loads(openai_response.choices[0].message.content)
json_response

{'name': 'David Nguyen',
 'major': 'computer science',
 'school': 'Stanford University',
 'grades': 3.8,
 'club': 'Robotics Club'}

# OpenAI function calling 

In [9]:
student_custom_functions = [
    {
        'name': 'extract_student_info',
        'description': 'Get the student information from the body of the input text',
        'parameters': {
            'type': 'object',
            'properties': {
                'name': {
                    'type': 'string',
                    'description': 'Name of the person'
                },
                'major': {
                    'type': 'string',
                    'description': 'Major subject.'
                },
                'school': {
                    'type': 'string',
                    'description': 'The university name.'
                },
                'grades': {
                    'type': 'integer',
                    'description': 'GPA of the student.'
                },
                'club': {
                    'type': 'string',
                    'description': 'School club for extracurricular activities. '
                }
                
            }
        }
    }
]

In [10]:
student_2_description="Ravi Patel is a sophomore majoring in computer science at the University of Michigan. He is South Asian Indian American and has a 3.7 GPA. Ravi is an active member of the university's Chess Club and the South Asian Student Association. He hopes to pursue a career in software engineering after graduating."
student_description = [student_1_description,student_2_description]


In [11]:
for i in student_description:
    
    response = client.chat.completions.create(
        model = 'gpt-3.5-turbo',
        messages = [{'role': 'user', 'content': i}],
        functions = student_custom_functions,
        function_call = 'auto'

    )
    json_response = json.loads(response.choices[0].message.function_call.arguments)
    print(json_response)

{}
{'name': 'Ravi Patel', 'major': 'computer science', 'school': 'University of Michigan', 'grades': 3.7, 'club': 'Chess Club'}


In [12]:
custom_functions = [
    {
        'name': 'extract_student_info',
        'description': 'Get the student information from the body of the input text',
        'parameters': {
            'type': 'object',
            'properties': {
                'name': {
                    'type': 'string',
                    'description': 'Name of the person'
                },
                'major': {
                    'type': 'string',
                    'description': 'Major subject.'
                },
                'school': {
                    'type': 'string',
                    'description': 'The university name.'
                },
                'grades': {
                    'type': 'integer',
                    'description': 'GPA of the student.'
                },
                'club': {
                    'type': 'string',
                    'description': 'School club for extracurricular activities. '
                }
                
            }
        }
    },
    {
        'name': 'extract_school_info',
        'description': 'Get the school information from the body of the input text',
        'parameters': {
            'type': 'object',
            'properties': {
                'name': {
                    'type': 'string',
                    'description': 'Name of the school.'
                },
                'ranking': {
                    'type': 'integer',
                    'description': 'QS world ranking of the school.'
                },
                'country': {
                    'type': 'string',
                    'description': 'Country of the school.'
                },
                'no_of_students': {
                    'type': 'integer',
                    'description': 'Number of students enrolled in the school.'
                }
            }
        }
    }
]

In [13]:
school_1_description = "Stanford University is a private research university located in Stanford, California, United States. It was founded in 1885 by Leland Stanford and his wife, Jane Stanford, in memory of their only child, Leland Stanford Jr. The university is ranked #5 in the world by QS World University Rankings. It has over 17,000 students, including about 7,600 undergraduates and 9,500 graduates23. "

In [14]:
description = [student_1_description, school_1_description]
for i in description:
    response = client.chat.completions.create(
        model = 'gpt-3.5-turbo',
        messages = [{'role': 'user', 'content': i}],
        functions = custom_functions,
        function_call = 'auto'
    )

    # Loading the response as a JSON object
    json_response = json.loads(response.choices[0].message.function_call.arguments)
    print(json_response)

{'name': 'David Nguyen', 'major': 'computer science', 'school': 'Stanford University', 'grades': 3.8, 'club': 'Robotics Club'}
{'name': 'Stanford University', 'ranking': 5, 'country': 'United States', 'no_of_students': 17000}


In [15]:
def extract_student_info(name, major, school, grades, club):
    
    """Get the student information"""

    return f"{name} is majoring in {major} at {school}. He has {grades} GPA and he is an active member of the university's {club}."

def extract_school_info(name, ranking, country, no_of_students):
    
    """Get the school information"""

    return f"{name} is located in the {country}. The university is ranked #{ranking} in the world with {no_of_students} students."

In [25]:
descriptions = [
    student_1_description, 
    "Who was a Abraham Lincoln?",
    school_1_description
                ]

for i, sample in enumerate(descriptions):
    response = client.chat.completions.create(
        model = 'gpt-3.5-turbo',
        messages = [{'role': 'user', 'content': sample}],
        functions = custom_functions,
        function_call = 'auto'
    )
    
    response_message = response.choices[0].message
    print(response_message)
    break
    if dict(response_message).get('function_call'):
        
        # Which function call was invoked
        function_called = response_message.function_call.name
        
        # Extracting the arguments
        function_args  = json.loads(response_message.function_call.arguments)
        
        # Function names
        available_functions = {
            "extract_school_info": extract_school_info,
            "extract_student_info": extract_student_info
        }
        
        fuction_to_call = available_functions[function_called]
        response_message = fuction_to_call(*list(function_args.values()))
        
    else:
        response_message = response_message.content
    
    print(f"\nSample#{i+1}\n")
    print(response_message)

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"name":"David Nguyen","major":"computer science","school":"Stanford University","grades":3.8,"club":"Robotics Club"}', name='extract_student_info'), tool_calls=None)


In [26]:
function_called = response_message.function_call.name
function_called

'extract_student_info'

In [27]:
function_args  = json.loads(response_message.function_call.arguments)


In [28]:

function_args

{'name': 'David Nguyen',
 'major': 'computer science',
 'school': 'Stanford University',
 'grades': 3.8,
 'club': 'Robotics Club'}

In [29]:
 available_functions = {
            "extract_school_info": extract_school_info,
            "extract_student_info": extract_student_info
        }
        

In [30]:
fuction_to_call = available_functions[function_called]

In [32]:
fuction_to_call(*list(function_args .values()))

"David Nguyen is majoring in computer science at Stanford University. He has 3.8 GPA and he is an active member of the university's Robotics Club."